In [219]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('../data/checking-logs.sqlite')


In [220]:
pd.read_sql('PRAGMA table_info(test);', conn)


,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [221]:
pd.read_sql('SELECT * FROM test LIMIT 10;', conn)


,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [222]:
query = """
SELECT uid, MIN(diff)
FROM
    (SELECT uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 as integer) as diff
    FROM test t
    LEFT JOIN deadlines dl on t.labname = dl.labs
    WHERE NOT t.labname = 'project1'
    ORDER BY 2)gg
LIMIT 1
"""

df_min = pd.io.sql.read_sql(query, conn)
df_min

,uid,MIN(diff)
0,user_30,-202


In [223]:
query = """
SELECT uid, MAX(diff)
FROM
    (SELECT uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 as integer) as diff
    FROM test t
    LEFT JOIN deadlines dl on t.labname = dl.labs
    WHERE NOT t.labname = 'project1'
    ORDER BY 2)gg
LIMIT 1
"""

df_max = pd.io.sql.read_sql(query, conn)
df_max

,uid,MAX(diff)
0,user_25,-2


In [224]:
query = """
SELECT AVG(diff) as avg_diff_hours
FROM (
    SELECT CAST((julianday(t.first_commit_ts) - julianday(datetime(d.deadlines,'unixepoch'))) * 24 AS INTEGER) as diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname <> 'project1'
);
"""
df_avg = pd.read_sql(query, conn)
df_avg

,avg_diff_hours
0,-89.125


In [225]:
views_diff_sql = """
WITH diffs AS (
    SELECT t.uid,
           /*AVG*/( (julianday(t.first_commit_ts) - julianday(datetime(d.deadlines,'unixepoch'))) * 24 ) AS avg_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname <> 'project1'
    GROUP BY t.uid
),
pv AS (
    SELECT uid, COUNT(*) AS pageviews
    FROM pageviews
    WHERE uid LIKE 'user_%'
    GROUP BY uid
)
SELECT d.uid, d.avg_diff, COALESCE(pv.pageviews,0) AS pageviews
FROM diffs d
LEFT JOIN pv ON d.uid = pv.uid;
"""
views_diff = pd.read_sql(views_diff_sql, conn)
num_corr = views_diff.corr(numeric_only=True)
print(num_corr)


           avg_diff  pageviews
avg_diff    1.00000   -0.06324
pageviews  -0.06324    1.00000


In [226]:
conn.close()
